# **Extract keyframes from videos with scene detection support from TransnetV2**

## **1. Install required packages**

In [1]:
!pip install ffmpeg-python pillow
!git clone https://github.com/soCzech/TransNetV2.git
%cd TransNetV2/inference

Cloning into 'TransNetV2'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 362 (delta 71), reused 71 (delta 71), pack-reused 274 (from 1)
Receiving objects: 100% (362/362), 95.25 KiB | 1.64 MiB/s, done.
Resolving deltas: 100% (210/210), done.
/kaggle/working/TransNetV2/inference


In [2]:
import os
import cv2
import json
import glob
import torch
import ffmpeg
import numpy as np
from tqdm.auto import tqdm
from transnetv2 import TransNetV2
from timeit import default_timer as timer

In [3]:
# Defined input directories
input_dirs = [
    # '/kaggle/input/aic2024-videos-part1-1', # Contains L01 to L06
    '/kaggle/input/aic2024-videos-part1'     # Contains L07 to L12
]

all_video_paths = dict()

# Iterate through both directories
for videos_dir in input_dirs:
    if not os.path.exists(videos_dir):
        print(f"Directory not found: {videos_dir}")
        continue
        
    for part in sorted(os.listdir(videos_dir)):
        # Check if folder name matches "Videos_Lxx" pattern
        if not part.startswith("Videos_"):
            continue
            
        data_part = part.split('_')[-1] # Extracts L01, L02...
        
        # Initialize dictionary for this part if not exists
        if data_part not in all_video_paths:
            all_video_paths[data_part] = dict()

        data_part_path = f'{videos_dir}/Videos_{data_part}/video'
        
        if not os.path.exists(data_part_path):
            continue
            
        video_paths = sorted(os.listdir(data_part_path))
        # Filter for mp4 only to be safe
        video_ids = [vp.replace('.mp4', '').split('_')[-1] for vp in video_paths if vp.endswith('.mp4')]
        
        for video_id, video_path in zip(video_ids, video_paths):
            if not video_path.endswith('.mp4'): continue
            video_path_full = f'{data_part_path}/{video_path}'
            all_video_paths[data_part][video_id] = video_path_full

print(f"Total Parts Found: {sorted(all_video_paths.keys())}")

Total Parts Found: ['L07', 'L08', 'L09', 'L10', 'L11', 'L12']


In [4]:
num_batch = 1
BATCH_ID = 0
MEMBER_ID = 0    
num_member = 1   # Chạy 100% video

all_videos = [x for v in all_video_paths.values() for x in v.values()]

# Safety check
if len(all_videos) > 0:
    # Tính số lượng video mỗi member cần làm
    # Dùng math.ceil để đảm bảo chia hết hoặc dư vào batch cuối
    import math
    batch_len = math.ceil(len(all_videos) / num_batch / num_member)
else:
    batch_len = 0

all_batches_info = {n: {} for n in range(num_batch)}
current_idx = 0

for n in range(num_batch):
    for m in range(num_member):
        start = current_idx
        end = current_idx + batch_len
        
        # Đảm bảo không vượt quá tổng số video
        if end > len(all_videos):
            end = len(all_videos)
            
        # --- FIX QUAN TRỌNG TẠI ĐÂY ---
        # Luôn gán vào dict con [m], KHÔNG BAO GIỜ gán trực tiếp list vào [n]
        # Ngay cả khi num_member = 1, ta vẫn dùng key [0]
        all_batches_info[n][m] = all_videos[start:end]
        
        current_idx = end
        
# Debug: Kiểm tra xem nó có phải là List không
print(f"Type check: {type(all_batches_info[BATCH_ID][MEMBER_ID])}") 
# Nó phải in ra <class 'list'> thì mới đúng. Trước đó nó in ra <class 'str'> nên mới lỗi.

with open("/kaggle/working/batch_info.json", 'w') as f:
    # Convert keys to str for JSON serialization if needed, though int keys are coerced
    json.dump(all_batches_info, f)
    
print(f"Total videos to process: {len(all_videos)}")
print(f"Videos assigned to this worker: {len(all_batches_info[BATCH_ID][MEMBER_ID])}")

Type check: <class 'list'>
Total videos to process: 179
Videos assigned to this worker: 179


## 3. **Extract shots**

In [5]:
model = TransNetV2()

[TransNetV2] Using weights from /kaggle/working/TransNetV2/inference/transnetv2-weights/.


In [6]:
%%time
import os
import json

save_dir = '/kaggle/working/scenes'
os.makedirs(save_dir, exist_ok=True)

# Lấy danh sách video từ config ở Cell 4
videos_to_process = all_batches_info[BATCH_ID][MEMBER_ID]
print(f"🚀 Start processing {len(videos_to_process)} videos for Scene Detection...")

count = 0
for i, video_path in enumerate(videos_to_process):
    try:
        # --- LOGIC FIX LỖI TÊN FILE ---
        filename = video_path.split('/')[-1]
        parts = filename.split('_')
        
        if len(parts) >= 2:
            # Chuẩn: L01_V001.mp4 -> Batch: L01
            video_batch = parts[0]
            video_name = "_".join(parts[1:])
        else:
            # Lỗi: test.mp4 -> Lấy Batch từ tên thư mục cha (Videos_L01)
            parent_dir = video_path.split('/')[-3] 
            if "Videos_" in parent_dir:
                video_batch = parent_dir.split('_')[-1]
            else:
                video_batch = "Uncategorized"
            video_name = filename

        video_name = video_name.replace('.mp4', '')
        
        # Tạo thư mục con: /scenes/L01
        batch_save_dir = os.path.join(save_dir, video_batch)
        os.makedirs(batch_save_dir, exist_ok=True)
        
        json_path = f"{batch_save_dir}/{video_name}.json"
        
        # Nếu file đã có rồi thì bỏ qua (để resume nếu bị ngắt)
        if os.path.exists(json_path):
            continue

        # --- GỌI TRANSNET MODEL ---
        _, single_frame_predictions, _ = model.predict_video(video_path)
        scenes = model.predictions_to_scenes(single_frame_predictions)
        
        with open(json_path, 'w') as f:
            json.dump(scenes.tolist(), f)
            
        count += 1
        if count % 10 == 0:
            print(f"   ...Processed {count}/{len(videos_to_process)} scenes.")

    except Exception as e:
        print(f"⚠️ Error on {video_path}: {e}")
        continue

print(f"✅ Scene Detection FINISHED! Created {count} json files.")

🚀 Start processing 179 videos for Scene Detection...
[TransNetV2] Extracting frames from /kaggle/input/aic2024-videos-part1/Videos_L07/video/L07_V001.mp4
[TransNetV2] Processing video frames 26542/26542
[TransNetV2] Extracting frames from /kaggle/input/aic2024-videos-part1/Videos_L07/video/L07_V002.mp4
[TransNetV2] Processing video frames 28065/28065
[TransNetV2] Extracting frames from /kaggle/input/aic2024-videos-part1/Videos_L07/video/L07_V003.mp4
[TransNetV2] Processing video frames 26629/26629
[TransNetV2] Extracting frames from /kaggle/input/aic2024-videos-part1/Videos_L07/video/L07_V004.mp4
[TransNetV2] Processing video frames 32688/32688
[TransNetV2] Extracting frames from /kaggle/input/aic2024-videos-part1/Videos_L07/video/L07_V005.mp4
[TransNetV2] Processing video frames 27074/27074
[TransNetV2] Extracting frames from /kaggle/input/aic2024-videos-part1/Videos_L07/video/L07_V006.mp4
[TransNetV2] Processing video frames 29282/29282
[TransNetV2] Extracting frames from /kaggle/inp

## **4. Extract frames from scenes**

In [7]:
def save_frames(video_path: str, frame_numbers: np.ndarray, save_dir: str):
    """
    Extract frames from a video using OpenCV. Optimized for logging.
    """
    video = cv2.VideoCapture(video_path)
    
    # Sort frame numbers to ensure sequential reading (faster)
    frame_numbers = np.sort(np.unique(frame_numbers))
    
    frame_idx = 0
    if len(frame_numbers) == 0:
        video.release()
        return

    frame_it = frame_numbers[frame_idx]
    # We only need to read up to the last required frame
    max_frame = frame_numbers[-1] + 1
    
    saved_count = 0
    
    # REMOVED tqdm here to prevent nested progress bars spamming
    for i in range(max_frame):
        ret, frame = video.read()       

        if not ret:
            # End of video reached prematurely
            break
            
        if i == frame_it:
            filename = "{}/{:0>4d}.jpg".format(f'{save_dir}', i) # Changed index to i for real frame ID
            
            if cv2.imwrite(filename, frame):
                saved_count += 1
            
            frame_idx += 1
            if frame_idx < len(frame_numbers):
                frame_it = frame_numbers[frame_idx]
            else:
                break

    video.release()

In [8]:
# --- CELL 8: Split Strategy & Safe Direct-Write ---
import numpy as np
from tqdm.auto import tqdm
import cv2
import os
import json
import zipfile
import shutil
import glob

# ==========================================
# ⚙️ CẤU HÌNH BATCH CẦN CHẠY TẠI ĐÂY
# ==========================================
# Run 1: Chạy L01 -> L05
TARGET_BATCHES = ['L07', 'L08', 'L09', 'L10']
OUTPUT_ZIP_NAME = 'keyframes_part2_set1.zip'

# Run 2 (Notebook khác hoặc lần chạy sau): Uncomment dòng dưới
# TARGET_BATCHES = ['L06'] 
# OUTPUT_ZIP_NAME = 'keyframes_part1_set2.zip'
# ==========================================

# Đường dẫn làm việc
work_dir = '/kaggle/working'
scene_json_dirs = '/kaggle/working/scenes'
final_zip_path = os.path.join(work_dir, OUTPUT_ZIP_NAME)

# Dọn dẹp file cũ nếu có để tránh lỗi append
if os.path.exists(final_zip_path):
    print(f"⚠️ Found existing {OUTPUT_ZIP_NAME}, removing to start fresh...")
    os.remove(final_zip_path)

# --- HÀM CHIẾN LƯỢC FRAME (GIỮ NGUYÊN) ---
def get_adaptive_frames(scenes):
    frames_to_capture = []
    for start, end in scenes:
        duration = end - start
        if duration <= 1: continue 
        if duration < 25: 
            frames_to_capture.append((start + end) // 2)
        elif duration < 150: 
            frames_to_capture.extend([start, (start + end) // 2, end - 1])
        else: 
            step = 50 
            sampled = range(start, end, step)
            frames_to_capture.extend(sampled)
            if (end - 1) not in frames_to_capture:
                frames_to_capture.append(end - 1)
    return sorted(list(set(frames_to_capture)))

# --- HÀM KIỂM TRA DUNG LƯỢNG TRỐNG ---
def get_free_space_gb():
    total, used, free = shutil.disk_usage(work_dir)
    return free / (1024**3)

# --- VÒNG LẶP CHÍNH ---
print(f"🚀 Starting Extraction for Batches: {TARGET_BATCHES}")
print(f"💾 Saving to: {OUTPUT_ZIP_NAME}")

# Mở file Zip MỘT LẦN DUY NHẤT ở chế độ 'w'
# Điều này giúp tối ưu I/O và tránh lỗi phân mảnh file
with zipfile.ZipFile(final_zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    
    stop_processing = False
    
    for key in TARGET_BATCHES:
        if stop_processing: break
        
        # Kiểm tra xem Batch này có trong dữ liệu đầu vào không
        if key not in all_video_paths:
            print(f"⚠️ Batch {key} not found in input sources. Skipping.")
            continue
            
        video_paths_dict = all_video_paths[key]
        video_ids = sorted(video_paths_dict.keys())
        
        print(f"👉 Processing Batch {key} ({len(video_ids)} videos)...")
        
        # Progress bar cho từng video
        pbar = tqdm(video_ids, desc=f"Zipping {key}", unit="vid")
        
        for video_id in pbar:
            # --- SAFETY CHECK: DỪNG NẾU Ổ CỨNG SẮP ĐẦY (< 1GB) ---
            if get_free_space_gb() < 1.0:
                print(f"\n🛑 CRITICAL WARNING: Disk space low ({get_free_space_gb():.2f} GB left).")
                print("🛑 Stopping gracefully to save current Zip file.")
                stop_processing = True
                break

            # Logic tìm file Scene JSON
            scene_path_v1 = f'{scene_json_dirs}/{key}/{key}_{video_id}.json'
            scene_path_v2 = f'{scene_json_dirs}/{key}/{video_id}.json'
            final_scene_path = scene_path_v1 if os.path.exists(scene_path_v1) else (scene_path_v2 if os.path.exists(scene_path_v2) else None)
            
            if not final_scene_path: continue

            try:
                with open(final_scene_path) as f:
                    scenes = json.load(f)
                scenes = np.array([list(row) for row in scenes])
                if len(scenes) == 0: continue

                frame_numbers = get_adaptive_frames(scenes)
                if not frame_numbers: continue
                
                # Mở Video & Extract thẳng vào RAM -> Zip
                video_path = video_paths_dict[video_id]
                vid_cap = cv2.VideoCapture(video_path)
                
                f_idx = 0
                max_f = frame_numbers[-1] + 1
                
                for i in range(max_f):
                    ret, frame = vid_cap.read()
                    if not ret: break
                    
                    if i == frame_numbers[f_idx]:
                        # Nén ảnh JPG vào RAM (Quality 80)
                        ret_enc, buffer = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
                        
                        if ret_enc:
                            # Cấu trúc: keyframes/L01/V001/0001.jpg
                            zip_entry_name = f"keyframes/{key}/{video_id}/{i:04d}.jpg"
                            zf.writestr(zip_entry_name, buffer.tobytes())
                        
                        f_idx += 1
                        if f_idx >= len(frame_numbers): break
                            
                vid_cap.release()
                
            except Exception as e:
                # print(f"Error {video_id}: {e}")
                pass

    if stop_processing:
        print("⚠️ Process stopped early due to disk limits.")
    else:
        print(f"✅ Finished processing all requested batches: {TARGET_BATCHES}")

print(f"📦 Closing Zip file...")
# Khi thoát khỏi block 'with', file zip sẽ tự động được finalize an toàn

final_size = os.path.getsize(final_zip_path) / (1024**3)
print(f"🎉 DONE! File created: {OUTPUT_ZIP_NAME} ({final_size:.2f} GB)")

🚀 Starting Extraction for Batches: ['L07', 'L08', 'L09', 'L10']
💾 Saving to: keyframes_part2_set1.zip
👉 Processing Batch L07 (31 videos)...


Zipping L07:   0%|          | 0/31 [00:00<?, ?vid/s]

👉 Processing Batch L08 (30 videos)...


Zipping L08:   0%|          | 0/30 [00:00<?, ?vid/s]

👉 Processing Batch L09 (29 videos)...


Zipping L09:   0%|          | 0/29 [00:00<?, ?vid/s]

👉 Processing Batch L10 (29 videos)...


Zipping L10:   0%|          | 0/29 [00:00<?, ?vid/s]

✅ Finished processing all requested batches: ['L07', 'L08', 'L09', 'L10']
📦 Closing Zip file...
🎉 DONE! File created: keyframes_part2_set1.zip (13.79 GB)


In [9]:
# %cd /kaggle/working/

# print("📦 Zipping Scenes...")
# !zip -rq scenes.zip scenes/

# import os
# def get_size(path):
#     if os.path.exists(path):
#         size = os.path.getsize(path) / (1024 * 1024 * 1024)
#         print(f"   -> {path}: {size:.2f} GB")
#     else:
#         print(f"   -> {path}: NOT FOUND")

# print("📊 Checking Output Sizes:")
# get_size('keyframes.zip')
# get_size('scenes.zip')

# from IPython.display import FileLink
# print("\n⬇️ DOWNLOAD LINKS:")
# display(FileLink('keyframes.zip'))
# display(FileLink('scenes.zip'))

# # Cleanup (Optional)
# # !rm -rf scenes
# # !rm -rf temp_keyframes